In [1]:
push!(LOAD_PATH, @__DIR__)

4-element Vector{String}:
 "@"
 "@v#.#"
 "@stdlib"
 "/home/jpsmolka/Studia/Semestr_IV/TC/Software"

In [2]:
using LogicalFunctions

In [3]:
function positive(table::TruthTable)::TruthTable
    filter(
        entry -> entry.second,
        table
    )
end

positive (generic function with 1 method)

In [4]:
f(a, b, c) = (a && b) || (!b && c) || c

f (generic function with 1 method)

In [5]:
table = evaluate(f, exhaust(3)) |> 
    positive |> 
    keys

KeySet for a Dict{BitVector, Union{Nothing, Bool}} with 5 entries. Keys:
  Bool[1, 0, 1]
  Bool[0, 0, 1]
  Bool[1, 1, 1]
  Bool[0, 1, 1]
  Bool[1, 1, 0]

In [6]:
table .|> neighbours

5-element Vector{Vector{Union{BitVector, Vector{Union{Missing, Bool}}}}}:
 [Bool[0, 0, 1], Bool[1, 1, 1], Bool[1, 0, 0]]
 [Bool[1, 0, 1], Bool[0, 1, 1], Bool[0, 0, 0]]
 [Bool[0, 1, 1], Bool[1, 0, 1], Bool[1, 1, 0]]
 [Bool[1, 1, 1], Bool[0, 0, 1], Bool[0, 1, 0]]
 [Bool[0, 1, 0], Bool[1, 0, 0], Bool[1, 1, 1]]

In [7]:
to_match = collect(table)

5-element Vector{BitVector}:
 [1, 0, 1]
 [0, 0, 1]
 [1, 1, 1]
 [0, 1, 1]
 [1, 1, 0]

In [8]:
using Base.Iterators: filter as lfilter

In [9]:
⟺(a::Bool, b::Bool) = a === b

⟺ (generic function with 1 method)

In [10]:
table = evaluate(f, exhaust(3)) |> 
    positive |> 
    keys |>
    Set

display(table)

processed = Dict(table .=> false)
matched = Set()

for vec in lfilter(v -> v in table, collect(table))
    for neighbour in lfilter(neigh -> neigh in table, neighbours(vec))
        if (neighbour, vec) ∉ matched
            push!(matched, (vec, neighbour))
            processed[vec] = true
        end
    end
end

display(matched)
display(processed)
display(table)

Set{BitVector} with 5 elements:
  Bool[1, 0, 1]
  Bool[0, 0, 1]
  Bool[1, 1, 1]
  Bool[0, 1, 1]
  Bool[1, 1, 0]

Set{Any} with 5 elements:
  (Bool[1, 0, 1], Bool[1, 1, 1])
  (Bool[1, 1, 1], Bool[0, 1, 1])
  (Bool[1, 1, 1], Bool[1, 1, 0])
  (Bool[1, 0, 1], Bool[0, 0, 1])
  (Bool[0, 0, 1], Bool[0, 1, 1])

Dict{BitVector, Bool} with 5 entries:
  [1, 0, 1] => 1
  [0, 0, 1] => 1
  [1, 1, 1] => 1
  [0, 1, 1] => 0
  [1, 1, 0] => 0

Set{BitVector} with 5 elements:
  Bool[1, 0, 1]
  Bool[0, 0, 1]
  Bool[1, 1, 1]
  Bool[0, 1, 1]
  Bool[1, 1, 0]

In [45]:
function match!(table)
    matched = Set()
    processed = Dict(table .=> false)

    for vec in lfilter(v -> v in table, collect(table))
        for neighbour in lfilter(neigh -> neigh in table, neighbours(vec))
            if (neighbour, vec) ∉ matched
                push!(matched, (vec, neighbour))
                processed[vec] = true
                processed[neighbour] = true
            end
        end
    end

    return matched, filter(entry -> !entry.second, processed) |> keys |> Set
end

match! (generic function with 1 method)

In [53]:
table = evaluate(f, exhaust(3)) |> 
    positive |> 
    keys |>
    Set

matched = match!(table)
display.(matched)

Set{Any} with 5 elements:
  (Bool[1, 0, 1], Bool[1, 1, 1])
  (Bool[1, 1, 1], Bool[0, 1, 1])
  (Bool[1, 1, 1], Bool[1, 1, 0])
  (Bool[1, 0, 1], Bool[0, 0, 1])
  (Bool[0, 0, 1], Bool[0, 1, 1])

Set{BitVector}()

(nothing, nothing)

In [42]:
function combine!((vec1, vec2)::Tuple)
    let ret = Vector{Union{Bool, Missing}}(copy(vec1))
        diff = (vec1 .!== vec2) |> 
            skipmissing |> 
            argmax

        ret[diff] = missing
        ret
    end
end

combine! (generic function with 1 method)

In [47]:
function reduce_terms(table)
    prime = Set()

    matched = [1]
    
    while length(matched) > 0
        matched, non_matched = match!(table)
        union!(prime, non_matched)
        table = Set(combine!.(matched))
    end

    return prime
end

reduce_terms (generic function with 1 method)

In [57]:
g(a, b, c) = (!a) ⊻ (b || c)

table = evaluate(g, exhaust(3)) |> 
    positive |> 
    keys |>
    Set

res = reduce_terms(table)

Set{Any} with 3 elements:
  Union{Missing, Bool}[true, missing, true]
  Union{Missing, Bool}[true, true, missing]
  Bool[0, 0, 0]